### Import libraries

In [344]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import VarianceThreshold, RFECV, SelectFromModel
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

import lazypredict
from lazypredict.Supervised import LazyRegressor

### Read data

In [345]:
df = pd.read_csv("data/housing_iteration_6_regression.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.00,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Clean the data

In [346]:
df = df.set_index("Id")
df.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.00,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.00,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.00,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.00,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.00,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [347]:
# Let's visualize missing values
missing_values_count = df.isnull().sum().sort_values(ascending=False)
missing_df = pd.DataFrame({'Column': missing_values_count.index, 'Missing Values': missing_values_count.values})

# Display the DataFrame
print(missing_df.head(10))

         Column  Missing Values
0        PoolQC            1453
1   MiscFeature            1406
2         Alley            1369
3         Fence            1179
4   FireplaceQu             690
5   LotFrontage             259
6   GarageYrBlt              81
7    GarageCond              81
8    GarageType              81
9  GarageFinish              81


Top six columns have too many missing values. drop them. 

In [348]:
df = df.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "MasVnrType", "FireplaceQu", "LotFrontage"], axis=1)
df.describe()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.00,1460.00,1460.00,1460.00,1460.00,1460.00,1452.00,1460.00,1460.00,1460.00,...,1460.00,1460.00,1460.00,1460.00,1460.00,1460.00,1460.00,1460.00,1460.00,1460.00
mean,56.90,10516.83,6.10,5.58,1971.27,1984.87,103.69,443.64,46.55,567.24,...,94.24,46.66,21.95,3.41,15.06,2.76,43.49,6.32,2007.82,180921.20
std,42.30,9981.26,1.38,1.11,30.20,20.65,181.07,456.10,161.32,441.87,...,125.34,66.26,61.12,29.32,55.76,40.18,496.12,2.70,1.33,79442.50
min,20.00,1300.00,1.00,1.00,1872.00,1950.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2006.00,34900.00
25%,20.00,7553.50,5.00,5.00,1954.00,1967.00,0.00,0.00,0.00,223.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,2007.00,129975.00
50%,50.00,9478.50,6.00,5.00,1973.00,1994.00,0.00,383.50,0.00,477.50,...,0.00,25.00,0.00,0.00,0.00,0.00,0.00,6.00,2008.00,163000.00
75%,70.00,11601.50,7.00,6.00,2000.00,2004.00,166.00,712.25,0.00,808.00,...,168.00,68.00,0.00,0.00,0.00,0.00,0.00,8.00,2009.00,214000.00
max,190.00,215245.00,10.00,9.00,2010.00,2010.00,1600.00,5644.00,1474.00,2336.00,...,857.00,547.00,552.00,508.00,480.00,738.00,15500.00,12.00,2010.00,755000.00


Note the three columns indicating the year the house was sold, built, remodified and added garage: ["YrSold", "YearBuilt", "YearRemodAdd", "GarageYrBlt"].
These variables, though capture the year, are interpreted different. 


*YrSold* ranges only between 2006 and 2010 and captures the *Great Recession* years, starting from 2008. --> I convert this to a categorical variable. 
The three remaining year variables have similar interpretations (higher values meaning higher sale price) and hence, I keep them as they are. 

In [349]:
df["YrSold"] = df["YrSold"].astype("object")
df["YrSold"].head()

Id
1    2008
2    2007
3    2008
4    2006
5    2008
Name: YrSold, dtype: object

Similarly, the column *MoSold* doesnt make senses on its own. Convert it to categorical variable by seasons. 

In [350]:
# Define a custom function to assign seasons based on the 'MoSold' values
def categorize_season(month_sold):
    if month_sold in [12, 1, 2]:
        return 'Winter'
    elif month_sold in [3, 4, 5]:
        return 'Spring'
    elif month_sold in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Apply the custom function to create the new categorical column
df['MoSold'] = df['MoSold'].apply(categorize_season)
df['MoSold'].value_counts()

Summer    609
Spring    451
Fall      231
Winter    169
Name: MoSold, dtype: int64

### Split the data

In [351]:
#Create X and y dataframes
X = df.drop(["SalePrice"], axis=1)
y=df["SalePrice"]
print(X.shape, y.shape)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
print(X_train.shape, X_test.shape)

(1460, 72) (1460,)
(1168, 72) (292, 72)


### Create model pipeline
Now we will create a pipeline to process the data, feature and model selection. Note that the data columns consists of numerical and categorical columns, and therefore, the data processing shall be conducted separately for different column types

#### Separate numerical and categorical columns

In [352]:
# Create numerical dataframe
X_train_num = X_train.select_dtypes(include="number")

# Create categorical dataframe
X_train_cat = X_train.select_dtypes(exclude="number")

In [353]:
# Confirm the datatypes contained with a quick function
def check_data_types(df):
    return df.dtypes.unique()

# Check data types for each DataFrame
data_types_cat = check_data_types(X_train_cat)
data_types_num = check_data_types(X_train_num)

# Print the results
print("Data types in X_train_cat:", data_types_cat)
print("Data types in X_train_num:", data_types_num)

Data types in X_train_cat: [dtype('O')]
Data types in X_train_num: [dtype('int64') dtype('float64')]


#### Create numerical and categorical pipelines
First, let's impute the missing data using *SimpleImputer()* strategies. And then apply *OneHotEncoder* to encode categorical columns

In [354]:
# Categorical encoder pipeline
categorical_encoder = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NaN')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown="ignore", min_frequency=0.03))
])

# Numerical pipeline
num_pipe = make_pipeline(SimpleImputer(strategy="mean"))

Use ColumnTransformer to transform the categorical and numerical pipelines separately.

In [355]:
# Full preprocessing: a ColumnTransformer with 2 branches: numeric & categorical
full_preprocessing = ColumnTransformer(
    transformers=[
        ("num_pipe", num_pipe, X_train_num.columns.tolist()),
        ("cat_pipe", categorical_encoder, X_train_cat.columns.tolist()),
    ]
)

full_preprocessing

ColumnTransformer(transformers=[('num_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 ['MSSubClass', 'LotArea', 'OverallQual',
                                  'OverallCond', 'YearBuilt', 'YearRemodAdd',
                                  'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
                                  'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
                                  '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
                                  'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
                                  'HalfBath', 'BedroomA...
                                  'LandContour', 'Utilities', 'LotConfig',
                                  'LandSlope', 'Neighborhood', 'Condition1',
                                  'Condition2', 'BldgType', 'HouseStyle',
                                  'RoofStyle', 'RoofMatl', 'Exterior1st',
                                  'Exterior2nd', 'ExterQual', 'ExterCond',
                                  'Foundation', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Heating', 'HeatingQC',
                                  'CentralAir', 'Electrical', 'KitchenQual',
                                  'Functional', ...])])

In [356]:
# Make a full pipeline to include scaling and feature selection
full_pipeline = make_pipeline(full_preprocessing,
                            MinMaxScaler(), # MinMaxScaler to scale the data
                            RFECV(RandomForestRegressor()))  # Applied Recursive Feature Elimination method for feature selection
full_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  ['MSSubClass', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalf...
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'Exterior1st', 'Exterior2nd',
                                                   'ExterQual', 'ExterCond',
                                                   'Foundation', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', ...])])),
                ('minmaxscaler', MinMaxScaler()),
                ('rfecv', RFECV(estimator=RandomForestRegressor()))])

## Trying feature selection methods and estimators

### Prepare the data

In [357]:
# The new_pipeline is defined as follows:
new_pipeline = make_pipeline(full_preprocessing, MinMaxScaler())

# Transform the training data using the pipeline
X_train_transformed = new_pipeline.fit_transform(X_train)

# Transform the test data using the pipeline
X_test_transformed = new_pipeline.transform(X_test)

# Get the column names after preprocessing
transformed_columns = new_pipeline.named_steps['columntransformer'].get_feature_names_out()

# Create new dataframes with the transformed data
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=transformed_columns)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=transformed_columns)

# Check the shape of pre- and post-transformation dataframes
print(X_train_transformed_df.shape, X_test_transformed_df.shape)

(1168, 191) (292, 191)


### Baseline

In [358]:
# Decision tree.
var_tree = DecisionTreeRegressor()
var_tree.fit(X_train_transformed_df, y_train)
var_tree_pred = var_tree.predict(X_test_transformed_df)
dt_r2 = r2_score(y_true = y_test,
                 y_pred = var_tree_pred)
dt_rmse = np.sqrt(mean_squared_error(y_test, var_tree_pred))

# Random forest regressor
var_rf = RandomForestRegressor()
var_rf.fit(X_train_transformed_df, y_train)
var_rf_pred = var_rf.predict(X_test_transformed_df)
rf_r2 = r2_score(y_test, var_rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, var_rf_pred))

# K-Nearest Neighbors.
var_knn = KNeighborsRegressor(n_neighbors=1)
var_knn.fit(X_train_transformed_df, y_train)
var_knn_pred = var_knn.predict(X_test_transformed_df)
knn_r2 = r2_score(y_true = y_test,
                 y_pred = var_knn_pred)
knn_rmse = np.sqrt(mean_squared_error(y_test, var_knn_pred))

# SGD regressor
var_sgd = SGDRegressor()
var_sgd.fit(X_train_transformed_df, y_train)
sgd_predictions = var_sgd.predict(X_test_transformed_df)
sgd_r2 = r2_score(y_true = y_test,
                 y_pred = sgd_predictions)
sgd_rmse = np.sqrt(mean_squared_error(y_test, sgd_predictions))

# Linear Regression
var_lr = LinearRegression()
var_lr.fit(X_train_transformed_df, y_train)
lr_predictions = var_lr.predict(X_test_transformed_df)
lr_r2 = r2_score(y_true = y_test,
                 y_pred = lr_predictions)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_predictions))

# Gradient Boosting Regressor
var_gb = GradientBoostingRegressor()
var_gb.fit(X_train_transformed_df, y_train)
gb_predictions = var_gb.predict(X_test_transformed_df)
gb_r2 = r2_score(y_true = y_test,
                 y_pred = gb_predictions)
gb_rmse = np.sqrt(mean_squared_error(y_test, gb_predictions))

# Gather the R2 scores in a single dataframe to make comparison with other models and feature selection methods
performances = pd.DataFrame({'decision_tree': dt_r2, "random_forest": rf_r2, "knn": knn_r2, "SGD": sgd_r2, "LinearRegression": lr_r2, "GradientBoosting": gb_r2},
                            index=['Baseline'])

# Gather RMSE in a single dataframe to make comparison with other models and feature selection methods
rmse_scores = pd.DataFrame({'decision_tree': dt_rmse,
                            'random_forest': rf_rmse,
                            'knn': knn_rmse,
                            'SGD': sgd_rmse,
                            'LinearRegression': lr_rmse,
                            'GradientBoosting': gb_rmse},
                           index=['Baseline'])

# Create LazyRegressor without fitting
lazy_regressor = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

# Fit and evaluate using LazyRegressor
models, predictions = lazy_regressor.fit(X_train_transformed_df, X_test_transformed_df, y_train, y_test)
print(models)

 71%|███████▏  | 30/42 [00:14<00:02,  4.17it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████| 42/42 [00:22<00:00,  1.88it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001744 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3242
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 177
[LightGBM] [Info] Start training from score 181356.966610
                                                              Adjusted R-Squared  \
Model                                                                              
GradientBoostingRegressor                                                   0.69   
XGBRegressor                                                                0.65   
ExtraTreesRegressor                                                         0.60   
HistGradientBoostingRegressor                                               0.56   
LGBMRegressor                                                               0.53   
RandomForestRegressor                                            

*LazyPredict* suggests that *GradientBoostingRegressor* is the best model with the highest *R2_score* and lowest *RMSE*.


### Variance Threshold method of feature selection

In [359]:
# Let's first look at the range and variance of the columns.
range_var_df = (pd.DataFrame({
                'Range': X_train_transformed_df.max() - X_train_transformed_df.min(),
                'Variance': X_train_transformed_df.var()})
                .sort_values(by='Variance'))
print(range_var_df.head(10))

# Chose a variance threshold. All features with a smaller variance than the `threshold` will be deleted from the dataset.
selector = VarianceThreshold(threshold=0.02)
X_train_var = selector.fit_transform(X_train_transformed_df)

# Let's check how many features were dropped:
print("shape before:", X_train_transformed_df.shape)
print("shape after:", X_train_var.shape)

# Apply the variance threshold to the scaled test set
X_test_var = selector.transform(X_test_transformed_df)

# Decision tree.
var_tree = DecisionTreeRegressor()
var_tree.fit(X_train_var, y_train)
var_tree_pred = var_tree.predict(X_test_var)
performances.loc["Variance", "decision_tree"] = r2_score(y_true = y_test,
                 y_pred = var_tree_pred)
rmse_scores.loc["Variance", "decision_tree"] = np.sqrt(mean_squared_error(y_test, var_tree_pred))

# Random forest regressor
var_rf = RandomForestRegressor()
var_rf.fit(X_train_transformed_df, y_train)
var_rf_pred = var_rf.predict(X_test_transformed_df)
performances.loc["Variance", "random_forest"] = r2_score(y_test, var_rf_pred)
rmse_scores.loc["Variance", "random_forest"] = np.sqrt(mean_squared_error(y_test, var_rf_pred))

# K-Nearest Neighbors.
var_knn = KNeighborsRegressor(n_neighbors=1)
var_knn.fit(X_train_var, y_train)
var_knn_pred = var_knn.predict(X_test_var)
performances.loc["Variance", "knn"] = r2_score(y_true = y_test,
                 y_pred = var_knn_pred)
rmse_scores.loc["Variance", "knn"] = np.sqrt(mean_squared_error(y_test, var_knn_pred))

# SGD regressor
var_sgd = SGDRegressor()
var_sgd.fit(X_train_var, y_train)
sgd_predictions = var_sgd.predict(X_test_var)
performances.loc["Variance", "SGD"] = r2_score(y_true = y_test,
                 y_pred = sgd_predictions)
rmse_scores.loc["Variance", "SGD"] = np.sqrt(mean_squared_error(y_test, sgd_predictions))

# Linear Regression
var_lr = LinearRegression()
var_lr.fit(X_train_var, y_train)
lr_predictions = var_lr.predict(X_test_var)
performances.loc["Variance", "LinearRegression"] = r2_score(y_true = y_test,
                 y_pred = lr_predictions)
rmse_scores.loc["Variance", "LinearRegression"] = np.sqrt(mean_squared_error(y_test, lr_predictions))

# Gradient Boosting Regressor
var_gb = GradientBoostingRegressor()
var_gb.fit(X_train_var, y_train)
gb_predictions = var_gb.predict(X_test_var)
performances.loc["Variance", "GradientBoosting"] = r2_score(y_true = y_test,
                 y_pred = gb_predictions)
rmse_scores.loc["Variance", "GradientBoosting"] = np.sqrt(mean_squared_error(y_test, gb_predictions))

# Create LazyRegressor without fitting
lazy_regressor = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

# Fit and evaluate using LazyRegressor
models, predictions = lazy_regressor.fit(X_train_var, X_test_var, y_train, y_test)
print(models)

                                        Range  Variance
cat_pipe__Utilities_AllPub               1.00      0.00
cat_pipe__Utilities_infrequent_sklearn   1.00      0.00
num_pipe__MiscVal                        1.00      0.00
num_pipe__LotArea                        1.00      0.00
num_pipe__3SsnPorch                      1.00      0.00
num_pipe__PoolArea                       1.00      0.00
cat_pipe__Street_Pave                    1.00      0.01
cat_pipe__Street_infrequent_sklearn      1.00      0.01
num_pipe__KitchenAbvGr                   1.00      0.01
num_pipe__LowQualFinSF                   1.00      0.01
shape before: (1168, 191)
shape after: (1168, 159)


 71%|███████▏  | 30/42 [00:13<00:03,  3.61it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


 98%|█████████▊| 41/42 [00:22<00:00,  1.92it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001521 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1786
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 159
[LightGBM] [Info] Start training from score 181356.966610


100%|██████████| 42/42 [00:22<00:00,  1.85it/s]

                                                             Adjusted R-Squared  \
Model                                                                             
GradientBoostingRegressor                                                  0.73   
ExtraTreesRegressor                                                        0.68   
XGBRegressor                                                               0.65   
RandomForestRegressor                                                      0.60   
LGBMRegressor                                                              0.58   
HistGradientBoostingRegressor                                              0.58   
BaggingRegressor                                                           0.56   
ElasticNet                                                                 0.51   
TweedieRegressor                                                           0.51   
AdaBoostRegressor                                                          0.48   
Pass

*LazyPredict* suggests that *GradientBoostingRegressor* is the best model with the highest *R2_score* and lowest *RMSE*.

### Recursive Feature Elimination (RFE) method of feature selection

In [360]:
# Use DecisionTreeRegressor for feature selection using RFECV
rfe_dt = RFECV(DecisionTreeRegressor(max_depth=10, min_samples_leaf=18))
X_train_rfe = rfe_dt.fit_transform(X_train_transformed_df, y_train)
X_test_rfe = X_test_transformed_df.loc[:, rfe_dt.support_]

# Use Decision Tree regressor to estimate the model
rf_model = DecisionTreeRegressor(max_depth=10, min_samples_leaf=18)
rf_model.fit(X_train_rfe, y_train)
rf_pred = rf_model.predict(X_test_rfe)
performances.loc["RFE", "decision_tree"] = r2_score(y_test, rf_pred)
rmse_scores.loc["RFE", "decision_tree"] = np.sqrt(mean_squared_error(y_test, rf_pred))

# Use RandomForestRegressor to estimate the model
rf_model = RandomForestRegressor(max_depth=10, min_samples_leaf=18)
rf_model.fit(X_train_rfe, y_train)
rf_pred = rf_model.predict(X_test_rfe)
performances.loc["RFE", "random_forest"] = r2_score(y_test, rf_pred)
rmse_scores.loc["RFE", "random_forest"] = np.sqrt(mean_squared_error(y_test, rf_pred))

# KNeighborsRegressor
rfe_knn = KNeighborsRegressor(n_neighbors=1)
rfe_knn.fit(X_train_rfe, y_train)
rfe_knn_pred = rfe_knn.predict(X_test_rfe)
performances.loc["RFE", "knn"] = r2_score(y_test, rfe_knn_pred)
rmse_scores.loc["RFE", "knn"] = np.sqrt(mean_squared_error(y_test, rfe_knn_pred))

# SGD regressor
rfe_sgd = SGDRegressor()
rfe_sgd.fit(X_train_rfe, y_train)
rfe_sgd_pred = rfe_sgd.predict(X_test_rfe)
performances.loc["RFE", "SGD"] = r2_score(y_test, rfe_sgd_pred)
rmse_scores.loc["RFE", "SGD"] = np.sqrt(mean_squared_error(y_test, rfe_sgd_pred))

# Linear regression
rfe_lr = LinearRegression()
rfe_lr.fit(X_train_rfe, y_train)
rfe_lr_pred = rfe_lr.predict(X_test_rfe)
performances.loc["RFE", "LinearRegression"] = r2_score(y_test, rfe_lr_pred)
rmse_scores.loc["RFE", "LinearRegression"] = np.sqrt(mean_squared_error(y_test, rfe_lr_pred))

# Gradient Boosting Regressor
rfe_gb = GradientBoostingRegressor()
rfe_gb.fit(X_train_rfe, y_train)
rfe_gb_pred = rfe_gb.predict(X_test_rfe)
performances.loc["RFE", "GradientBoosting"] = r2_score(y_test, rfe_gb_pred)
rmse_scores.loc["RFE", "GradientBoosting"] = np.sqrt(mean_squared_error(y_test, rfe_gb_pred))

# Create LazyRegressor without fitting
lazy_regressor = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

# Fit and evaluate using LazyRegressor
models, predictions = lazy_regressor.fit(X_train_rfe, X_test_rfe, y_train, y_test)
print(models)

 67%|██████▋   | 28/42 [00:02<00:01, 10.84it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████| 42/42 [00:03<00:00, 10.51it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 526
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 4
[LightGBM] [Info] Start training from score 181356.966610
                               Adjusted R-Squared  R-Squared        RMSE  \
Model                                                                      
GradientBoostingRegressor                    0.81       0.81    33431.88   
ExtraTreesRegressor                          0.79       0.80    34748.70   
BaggingRegressor                             0.79       0.79    35292.19   
RandomForestRegressor                        0.79       0.79    35479.36   
XGBRegressor                                 0.78       0.78    35840.57   
AdaBoostRegressor                            0.78       0.78    36105.73   
HistGradientBoostingRegressor                0.7

*LazyPredict* suggests that *GradientBoostingRegressor* is the best model with the highest *R2_score* and lowest *RMSE*.

### SelectFromModel of feature selection

In [361]:
# Use DecisionTreeRegressor for feature selection using RFECV
select_model_tree = SelectFromModel(DecisionTreeRegressor(max_depth=10, min_samples_leaf=18),
                                    threshold=None)
# Transform the train set.
X_train_rfe = select_model_tree.fit_transform(X_train_transformed_df, y_train)

# Transform the test set.
X_test_rfe = select_model_tree.transform(X_test_transformed_df)

# Use Decision Tree regressor to estimate the model
rf_model = DecisionTreeRegressor(max_depth=10, min_samples_leaf=18)
rf_model.fit(X_train_rfe, y_train)
rf_pred = rf_model.predict(X_test_rfe)
performances.loc["Select_model", "decision_tree"] = r2_score(y_test, rf_pred)
rmse_scores.loc["Select_model", "decision_tree"] = np.sqrt(mean_squared_error(y_test, rf_pred))

# Use RandomForestRegressor to estimate the model
rf_model = RandomForestRegressor(max_depth=10, min_samples_leaf=18)
rf_model.fit(X_train_rfe, y_train)
rf_pred = rf_model.predict(X_test_rfe)
performances.loc["Select_model", "random_forest"] = r2_score(y_test, rf_pred)
rmse_scores.loc["Select_model", "random_forest"] = np.sqrt(mean_squared_error(y_test, rf_pred))

# KN regressor
rfe_knn = KNeighborsRegressor(n_neighbors=1)
rfe_knn.fit(X_train_rfe, y_train)
rfe_knn_pred = rfe_knn.predict(X_test_rfe)
performances.loc["Select_model", "knn"] = r2_score(y_test, rfe_knn_pred)
rmse_scores.loc["Select_model", "knn"] = np.sqrt(mean_squared_error(y_test, rfe_knn_pred))

# SGD regressor
rfe_sgd = SGDRegressor()
rfe_sgd.fit(X_train_rfe, y_train)
rfe_sgd_pred = rfe_sgd.predict(X_test_rfe)
performances.loc["Select_model", "SGD"] = r2_score(y_test, rfe_sgd_pred)
rmse_scores.loc["Select_model", "SGD"] = np.sqrt(mean_squared_error(y_test, rfe_sgd_pred))

# Linear regression
rfe_lr = LinearRegression()
rfe_lr.fit(X_train_rfe, y_train)
rfe_lr_pred = rfe_lr.predict(X_test_rfe)
performances.loc["Select_model", "LinearRegression"] = r2_score(y_test, rfe_lr_pred)
rmse_scores.loc["Select_model", "LinearRegression"] = np.sqrt(mean_squared_error(y_test, rfe_lr_pred))

# Gradient boosting regressor
rfe_gb = GradientBoostingRegressor()
rfe_gb.fit(X_train_rfe, y_train)
rfe_gb_pred = rfe_gb.predict(X_test_rfe)
performances.loc["Select_model", "GradientBoosting"] = r2_score(y_test, rfe_gb_pred)
rmse_scores.loc["Select_model", "GradientBoosting"] = np.sqrt(mean_squared_error(y_test, rfe_gb_pred))

# Create LazyRegressor without fitting
lazy_regressor = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

# Fit and evaluate using LazyRegressor
models, predictions = lazy_regressor.fit(X_train_rfe, X_test_rfe, y_train, y_test)
print(models)

 79%|███████▊  | 33/42 [00:04<00:00, 11.83it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████| 42/42 [00:06<00:00,  6.32it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001817 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1390
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 9
[LightGBM] [Info] Start training from score 181356.966610
                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
GradientBoostingRegressor                    0.87       0.87  27879.56   
XGBRegressor                                 0.86       0.87  28135.87   
ExtraTreesRegressor                          0.85       0.85  29679.75   
RandomForestRegressor                        0.83       0.84  31213.57   
BaggingRegressor                             0.83       0.83  31518.29   
LGBMRegressor                                0.81       0.82  32786.20   
HistGradientBoostingRegressor                0.81       0.82  3

*LazyPredict* consistently suggests *GradientBoostingRegressor* to be the best model with the highest *R2_score* and lowest *RMSE*.

In [362]:
performances

,decision_tree,random_forest,knn,SGD,LinearRegression,GradientBoosting
Baseline,0.75,0.82,0.54,0.73,-91633769830375456768.00,0.89
Variance,0.70,0.83,0.53,0.77,-18097183833429819392.00,0.88
RFE,0.72,0.78,0.72,0.64,0.62,0.81
Select_model,0.74,0.79,0.72,0.61,0.60,0.87


In [363]:
rmse_scores

,decision_tree,random_forest,knn,SGD,LinearRegression,GradientBoosting
Baseline,38818.08,32516.38,52369.45,40127.12,738581766080010.88,25328.22
Variance,42144.13,31676.31,52685.75,37056.36,328228511887592.38,27247.62
RFE,40814.32,35889.30,40542.86,46503.78,47380.47,33442.74
Select_model,39001.67,35088.75,40763.96,47976.54,48891.81,27939.21


*Baseline* and *GradientBoosting* models seem to perform better with *Select from model* approach toward feature selection. We will apply them in the next step.

### Create a new pipeline with Gradient Boosting Regressor

In [364]:
# The new_pipeline is defined as follows:
new_pipeline = make_pipeline(full_preprocessing, MinMaxScaler(), 
                             SelectFromModel(DecisionTreeRegressor(max_depth=10, min_samples_leaf=18), threshold=None, max_features=70), 
                             GradientBoostingRegressor())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "gradientboostingregressor__n_estimators": [1000, 1500, 2000], #, 500],
    "gradientboostingregressor__learning_rate": [0.08, 0.01, 0.012],
    "gradientboostingregressor__max_depth": [2, 3, 4],
    }

search = GridSearchCV(new_pipeline, 
                      param_grid, scoring='neg_root_mean_squared_error', cv=5,
                      verbose=1, n_jobs=-1)

search.fit(X_train, y_train)

print(search.best_params_)
print(search.best_score_)

y_pred_new = search.predict(X_test)
print(r2_score(y_test, y_pred_new))
print(np.sqrt(mean_squared_error(y_test, y_pred_new)))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'columntransformer__num_pipe__simpleimputer__strategy': 'median', 'gradientboostingregressor__learning_rate': 0.012, 'gradientboostingregressor__max_depth': 4, 'gradientboostingregressor__n_estimators': 1000}
-32236.8036152802
0.879674727556449
26763.889712955337


## Predict for Kaggle competition
### Prepare the Kaggle data

In [365]:
# Read the Kaggle test/validation data
url = "https://drive.google.com/file/d/1oxFATWwyRN7HUFmmvTzRrvzQy1panjQo/view?usp=drive_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
val_data_full = pd.read_csv(path)

# Set Id as index
val_data_full = val_data_full.set_index("Id")
val_data_full.head(5)

# Drop columns with missing values
val_data_full = val_data_full.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "MasVnrType", "FireplaceQu", "LotFrontage"], axis=1)
val_data_full.describe()
val_data_full["YrSold"] = val_data_full["YrSold"].astype("object")


# Define a custom function to assign seasons based on the 'MoSold' values
def categorize_season(month_sold):
    if month_sold in [12, 1, 2]:
        return 'Winter'
    elif month_sold in [3, 4, 5]:
        return 'Spring'
    elif month_sold in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Apply the custom function to create the new categorical column
val_data_full['MoSold'] = val_data_full['MoSold'].apply(categorize_season)
val_data_full['MoSold'].value_counts()

Summer    573
Spring    454
Fall      242
Winter    190
Name: MoSold, dtype: int64

### Predict using the model above

In [366]:
# Let's predict on the validation data
prediction_val = search.predict(val_data_full)
val_data_full["SalePrice"] = prediction_val
val_data_full=val_data_full.reset_index()
submission = val_data_full[["Id", "SalePrice"]]
submission.to_csv("submission.csv", index=False)

The RMSE score for the competition test data: 0.15467


### Without feature selection (Baseline prediction)

In [367]:
# The new_pipeline is defined as follows:
pipeline_1 = make_pipeline(full_preprocessing, MinMaxScaler(), 
                             # SelectFromModel(DecisionTreeRegressor(max_depth=10, min_samples_leaf=18), threshold=None, max_features=70), 
                             GradientBoostingRegressor())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "gradientboostingregressor__n_estimators": [1000, 1500, 2000], #, 500],
    "gradientboostingregressor__learning_rate": [0.08, 0.01, 0.012],
    "gradientboostingregressor__max_depth": [2, 3, 4],
    }

search_2 = GridSearchCV(pipeline_1, 
                      param_grid, scoring='neg_root_mean_squared_error', cv=5,
                      verbose=1, n_jobs=-1)

search_2.fit(X_train, y_train)

print(search_2.best_params_)
print(search_2.best_score_)

y_pred_new = search_2.predict(X_test)
print(r2_score(y_test, y_pred_new))
print(np.sqrt(mean_squared_error(y_test, y_pred_new)))


# Let's predict on the validation data
val_data_full = val_data_full.drop("SalePrice", axis=1)

prediction_val = search_2.predict(val_data_full)
val_data_full["SalePrice"] = prediction_val
val_data_full=val_data_full.reset_index()
submission = val_data_full[["Id", "SalePrice"]]
submission.to_csv("submission_baseline.csv", index=False)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'columntransformer__num_pipe__simpleimputer__strategy': 'mean', 'gradientboostingregressor__learning_rate': 0.012, 'gradientboostingregressor__max_depth': 3, 'gradientboostingregressor__n_estimators': 2000}
-27585.86247211087
0.8915778990457714
25405.615093416644


The RMSE score for the competition test data: 0.12849 (Best score of the lot).


### Drop all parameters from the DecisionTreeRegressor() in SelectFromModel()

In [368]:
# The new_pipeline is defined as follows:
pipeline_2 = make_pipeline(full_preprocessing, MinMaxScaler(), 
                             SelectFromModel(DecisionTreeRegressor(), threshold=None), 
                             GradientBoostingRegressor())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "gradientboostingregressor__n_estimators": [1000, 1500, 2000], #, 500],
    "gradientboostingregressor__learning_rate": [0.08, 0.01, 0.012],
    "gradientboostingregressor__max_depth": [2, 3, 4],
    }

search_3 = GridSearchCV(pipeline_2, 
                      param_grid, scoring='neg_root_mean_squared_error', cv=5,
                      verbose=1, n_jobs=-1)

search_3.fit(X_train, y_train)

print(search_3.best_params_)
print(search_3.best_score_)

y_pred_new = search_3.predict(X_test)

# Print metrics
print(r2_score(y_test, y_pred_new))
print(np.sqrt(mean_squared_error(y_test, y_pred_new)))


# Let's predict on the validation data
val_data_full = val_data_full.drop("SalePrice", axis=1)

prediction_val = search_3.predict(val_data_full)
val_data_full["SalePrice"] = prediction_val
val_data_full=val_data_full.reset_index()
submission = val_data_full[["Id", "SalePrice"]]
submission.to_csv("submission_3.csv", index=False)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'columntransformer__num_pipe__simpleimputer__strategy': 'mean', 'gradientboostingregressor__learning_rate': 0.01, 'gradientboostingregressor__max_depth': 4, 'gradientboostingregressor__n_estimators': 1000}
-30086.942501547466
0.8814453430644812
26566.241349728054


The RMSE score for the competition test date: 0.15594.


### Change the estimator for SelectFromModel

In [370]:
# The new_pipeline is defined as follows:
pipeline_3 = make_pipeline(full_preprocessing, MinMaxScaler(), 
                             SelectFromModel(RandomForestRegressor(), threshold=None), 
                             GradientBoostingRegressor())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "gradientboostingregressor__n_estimators": [1000, 1500, 2000], #, 500],
    "gradientboostingregressor__learning_rate": [0.08, 0.01, 0.012],
    "gradientboostingregressor__max_depth": [2, 3, 4],
    }

search_4 = GridSearchCV(pipeline_3, 
                      param_grid, scoring='neg_root_mean_squared_error', cv=5,
                      verbose=1, n_jobs=-1)

search_4.fit(X_train, y_train)

print(search_4.best_params_)
print(search_4.best_score_)

y_pred_new = search_4.predict(X_test)

# Print metrics
print(r2_score(y_test, y_pred_new))
print(np.sqrt(mean_squared_error(y_test, y_pred_new)))


val_data_full = pd.read_csv(path)

# Set Id as index
val_data_full = val_data_full.set_index("Id")
val_data_full.head(5)

# Drop columns with missing values
val_data_full = val_data_full.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "MasVnrType", "FireplaceQu", "LotFrontage"], axis=1)
val_data_full.describe()
val_data_full["YrSold"] = val_data_full["YrSold"].astype("object")


# Define a custom function to assign seasons based on the 'MoSold' values
def categorize_season(month_sold):
    if month_sold in [12, 1, 2]:
        return 'Winter'
    elif month_sold in [3, 4, 5]:
        return 'Spring'
    elif month_sold in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Apply the custom function to create the new categorical column
val_data_full['MoSold'] = val_data_full['MoSold'].apply(categorize_season)
val_data_full['MoSold'].value_counts()

# val_data_full = val_data_full.drop("SalePrice", axis=1)

# Let's predict on the validation data
prediction_val = search_4.predict(val_data_full)

val_data_full["SalePrice"] = prediction_val
val_data_full=val_data_full.reset_index()
submission = val_data_full[["Id", "SalePrice"]]
submission.to_csv("submission_4.csv", index=False)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'columntransformer__num_pipe__simpleimputer__strategy': 'median', 'gradientboostingregressor__learning_rate': 0.01, 'gradientboostingregressor__max_depth': 2, 'gradientboostingregressor__n_estimators': 1500}
-29720.455589233123
0.8633689140578114
28519.740187367726


The RMSE score for the competition test data: 0.16002. 


In summary, the baseline model gave the best results.